In [189]:
import json
from config import settings
import os


import polars as pl
from polars import col as c
from polars import selectors as cs
import networkx as nx
from networkx_function import generate_nx_edge
from typing_extensions import Union
import numpy as np
import scipy as sp

# from distflow_algorithm import DistFlow
import time
import pandapower as pp
import pandapower.networks as pn
from polars_function import (
    get_transfo_admittance,
    get_transfo_impedance,
    get_transfo_conductance,
    get_transfo_imaginary_component,
)

from data_connector import pandapower_to_distflow
from general_function import duckdb_to_dict, dict_to_duckdb, pl_to_dict
from networkx_function import get_all_edge_data, generate_shortest_path_length_matrix
import graphblas as gb
from utility.parser_utility import duckdb_to_changes_schema

from distflow_schema import DistFlowSchema

# Useless outside jupiternotebook because in settings.py a line that changes the directory to src for ipynb
os.chdir(os.getcwd().replace("/src", ""))

## Import Schema from duckdb

In [190]:
file_names: dict[str, str] = json.load(open(settings.INPUT_FILE_NAMES))
changes_schema = duckdb_to_changes_schema(file_path=file_names["duckdb_output"])
list(changes_schema.__dict__.keys())

Read and validate tables from matlab_grid.db file: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.51s/it]


['heartbeat',
 'resource',
 'equipment',
 'terminal',
 'busbar_section',
 'branch',
 'branch_parameter_event',
 'geo_event',
 'switch',
 'switch_event',
 'transformer',
 'transformer_end',
 'transformer_parameter_event',
 'tap',
 'tap_event',
 'bess',
 'energy_consumer',
 'external_network',
 'generating_unit',
 'container',
 'client',
 'substation',
 'base_voltage',
 'connectivity_node',
 'connectivity',
 'measurement',
 'measurement_point',
 'measurement_span']

## Create resource data

In [214]:
resource_data = changes_schema.resource.filter(
    (c("concrete_class") == "branch")
    | (c("concrete_class") == "switch")
    | (c("concrete_class") == "tranfsormer")
    | (c("concrete_class") == "external_network")
).select(
    c("uuid"), c("dso_code").alias("element_id"), c("concrete_class").alias("type")
)

## Add row index to connectivity_node to give the number of node
## Join connectivity_node with connectivity to get the eq_fk
## Create the connectivity_node dictionnary with side+eq_fk as key and node index as value
connectivity_node: dict[float, str] = pl_to_dict(
    changes_schema.connectivity_node.with_row_index()
    .join(changes_schema.connectivity, left_on="uuid", right_on="cn_fk", how="left")
    .select((c("side") + c("eq_fk")).alias("eq_fk_side"), c("index"))
)

## Add node from and node to for each edge with side+eq_fk
resource_data = resource_data.with_columns(
    ("t1" + c("uuid"))
    .replace_strict(connectivity_node, default=None)
    .alias(
        "u_of_edge"
    ),  # Replace side+eq_fk with node number from connectivity for equipment
    ("t2" + c("uuid"))
    .replace_strict(connectivity_node, default=None)
    .alias("v_of_edge"),
)
# Add branch parameter
branch = changes_schema.branch_parameter_event[["uuid", "eq_fk", "r", "x", "b", "g"]]

# Add branch parameter to line_data
resource_data = resource_data.join(
    branch, left_on="uuid", right_on="eq_fk", how="left"
).drop("uuid_right")
## Add n_tranfo to 1, useless, because automatically added by add_table from class DistFlowSchema and by default value is 1
# resource_data = resource_data.with_columns(pl.lit(1).alias("n_tranfo"))

## Search slack node id for DisFlowSchema
slack_node_id = resource_data.filter(c("type") == "external_network")[
    "u_of_edge"
].item()

## Remove the external network from the resource_data
resource_data = resource_data.filter(c("type") != "external_network")

## Transform resource_data in pu

In [215]:
## Transfo in pu
S_base = 1e6  # VA -> 1 MVA for distribution grid

U_b = changes_schema.base_voltage["nominal_voltage"].item()  # V
I_b = S_base / (3**0.5 * U_b)  # A
Z_b = U_b**2 / S_base  # Ohm
B_b = 1 / Z_b  # S


pu_base_changes_schema = {
    "U_b": U_b,
    "I_b": I_b,
    "Z_b": Z_b,
    "B_b": B_b,
    "S_base": S_base,
}

resource_data_pu = resource_data.with_columns(
    (c("g") * pu_base_changes_schema["Z_b"]).alias("g_pu"),
    (c("r") / pu_base_changes_schema["Z_b"]).alias("r_pu"),
    (c("x") / pu_base_changes_schema["Z_b"]).alias("x_pu"),
    (c("b") / pu_base_changes_schema["B_b"]).alias("b_pu"),
).drop(["r", "x", "b", "g"])

In [216]:
resource_data_pu

uuid,element_id,type,u_of_edge,v_of_edge,g_pu,r_pu,x_pu,b_pu
str,str,str,i64,i64,f64,f64,f64,f64
"""b4fdf239-cfe2-5453-91c3-84c1b2…","""line_1""","""branch""",1,0,0.0,0.019375,0.011297,7.9168e-7
"""3edce6fc-e164-5b3e-9b0d-8a1c86…","""line_2""","""branch""",2,0,0.0,0.204688,0.114625,0.000009
"""c173d7f3-f7ca-5f44-98d8-63c1aa…","""line_4""","""branch""",4,1,0.0,0.303,0.00911,2.2696e-7
"""c33afba9-0379-5d6a-9ca1-743cf7…","""line_5""","""branch""",5,1,0.0,0.00775,0.004519,3.1667e-7
"""020bd95c-9320-5560-9627-4bbe19…","""line_6""","""branch""",6,2,0.0,0.041406,0.0231875,0.000002
…,…,…,…,…,…,…,…,…
"""34edd04e-a827-5cf8-b233-43d285…","""line_55""","""branch""",55,54,0.0,0.005425,0.003163,2.2167e-7
"""9b34c19a-8b3e-54c6-9cd9-834bb5…","""line_56""","""branch""",56,54,0.0,0.3219375,0.009679,2.4114e-7
"""ec28307e-8860-52db-9611-713cad…","""line_57""","""branch""",57,55,0.0,0.2461875,0.007402,1.8440e-7


In [217]:
test = DistFlowSchema()
test.add_table(edge_data=resource_data_pu)

DistFlowSchema(edge_data=shape: (57, 8)
┌───────────┬───────────┬───────────┬───────────┬───────────┬──────┬───────────┬────────┐
│ u_of_edge ┆ v_of_edge ┆ r_pu      ┆ x_pu      ┆ b_pu      ┆ g_pu ┆ n_transfo ┆ type   │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---  ┆ ---       ┆ ---    │
│ i32       ┆ i32       ┆ f64       ┆ f64       ┆ f64       ┆ f64  ┆ f64       ┆ str    │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪══════╪═══════════╪════════╡
│ 1         ┆ 0         ┆ 0.019375  ┆ 0.011297  ┆ 7.9168e-7 ┆ 0.0  ┆ 1.0       ┆ branch │
│ 2         ┆ 0         ┆ 0.204688  ┆ 0.114625  ┆ 0.000009  ┆ 0.0  ┆ 1.0       ┆ branch │
│ 4         ┆ 1         ┆ 0.303     ┆ 0.00911   ┆ 2.2696e-7 ┆ 0.0  ┆ 1.0       ┆ branch │
│ 5         ┆ 1         ┆ 0.00775   ┆ 0.004519  ┆ 3.1667e-7 ┆ 0.0  ┆ 1.0       ┆ branch │
│ 6         ┆ 2         ┆ 0.041406  ┆ 0.0231875 ┆ 0.000002  ┆ 0.0  ┆ 1.0       ┆ branch │
│ …         ┆ …         ┆ …         ┆ …         ┆ …         

In [ ]:
generate_full_jacobian_matrix(distflow_schema: DistFlowSchema, slack_node_id: int)

In [ ]:
## u, v and unpivot
# line_data.pivot(index="eq_class", on="side", values="node_number")
## add r,x,b,g in pu

## import type (switch, branch, transformer) from table

## add n_trafo
# distflow_data = line_data.filter().pivot.....

In [ ]:
# edge_data: pl.DataFrame = line_data
# u_of_edge: int = pt.Field(dtype=pl.Int32)
# v_of_edge: int = pt.Field(dtype=pl.Int32)
# r_pu: float = pt.Field(dtype=pl.Float64, default=0.0)
# x_pu: float = pt.Field(dtype=pl.Float64, default=0.0)
# b_pu: float = pt.Field(dtype=pl.Float64, default=0.0)
# g_pu: float = pt.Field(dtype=pl.Float64, default=0.0)
# n_transfo: float = pt.Field(dtype=pl.Float64, default=1.0)
# type: TYPES = pt.Field(dtype=pl.Utf8, constraints=literal_constraint(pt.field, TYPES))

## Filter data and create distflowschema

TYPES = Literal["branch", "transformer", "switch"]
class EdgeData(pt.Model):
    u_of_edge: int = pt.Field(dtype=pl.Int32)
    v_of_edge: int = pt.Field(dtype=pl.Int32)
    r_pu: float = pt.Field(dtype=pl.Float64, default=0.0)
    x_pu: float = pt.Field(dtype=pl.Float64, default=0.0)
    b_pu: float = pt.Field(dtype=pl.Float64, default=0.0)
    g_pu: float = pt.Field(dtype=pl.Float64, default=0.0)
    n_transfo: float = pt.Field(dtype=pl.Float64, default=1.0)
    type: TYPES = pt.Field(dtype=pl.Utf8, constraints=literal_constraint(pt.field, TYPES))

edge_data

In [ ]:
# edge_data = 1
# test = DistFlowSchema(edge_data)

In [ ]:
# edge_data = test.edge_data
changes_schema.resource

diff,uuid,start_heartbeat,end_heartbeat,start,end,dso_code,concrete_class,name,feeder_fk,metadata,owner
str,str,"datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]",str,str,str,str,str,str
"""+""","""b4fdf239-cfe2-5453-91c3-84c1b2…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_1""","""branch""",null,null,null,null
"""+""","""3edce6fc-e164-5b3e-9b0d-8a1c86…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_2""","""branch""",null,null,null,null
"""+""","""c173d7f3-f7ca-5f44-98d8-63c1aa…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_4""","""branch""",null,null,null,null
"""+""","""c33afba9-0379-5d6a-9ca1-743cf7…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_5""","""branch""",null,null,null,null
"""+""","""020bd95c-9320-5560-9627-4bbe19…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_6""","""branch""",null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…
"""+""","""b2c29241-7bfa-4515-aebc-07b62b…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""node_number_54""","""energy_consumer""",null,null,null,null
"""+""","""4e3a5eb1-0a0b-40f7-9995-043be5…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""node_number_57""","""energy_consumer""",null,null,null,null
"""+""","""c8a03e72-a228-5007-ad70-3cc3ab…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_3""","""switch""",null,null,null,null


switch

branch

branch_parameter_event

In [ ]:
line_data = changes_schema.resource.filter(
    (c("concrete_class") == "branch") | (c("concrete_class") == "switch")
).select(c("uuid"), c("dso_code").alias("line_id"), c("concrete_class").alias("type"))
print(line_data)
cn_mapping = pl_to_dict(
    changes_schema.connectivity.select(
        pl.concat_str("eq_fk", "side").alias("id"), "uuid"
    )
)
## Add uuid of t1 and t2 for resource
line_data = line_data.with_columns(
    (c("uuid") + "t1")
    .replace_strict(cn_mapping, default=None)
    .alias("u_of_edge"),  # Replace eq_fk with uuid from connectivity for branch
    (c("uuid") + "t2").replace_strict(cn_mapping, default=None).alias("v_of_edge"),
)

# Add branch parameter
branch = changes_schema.branch_parameter_event[["uuid", "eq_fk", "r", "x", "b", "g"]]
# changes_schema.branch_parameter_event.select(
#     c("uuid"),
#     # c("source_fk"),
#     c("eq_fk"),
#     c("r"),
#     c("r0"),
#     c("x"),
#     c("x0"),
#     c("b"),
#     c("b0"),
#     c("g"),
#     c("g0"),
# )
# Add branch parameter to line_data
line_data = line_data.join(branch, left_on="uuid", right_on="eq_fk", how="left").drop(
    "uuid_right"
)

shape: (57, 3)
┌─────────────────────────────────┬─────────┬────────┐
│ uuid                            ┆ line_id ┆ type   │
│ ---                             ┆ ---     ┆ ---    │
│ str                             ┆ str     ┆ str    │
╞═════════════════════════════════╪═════════╪════════╡
│ b4fdf239-cfe2-5453-91c3-84c1b2… ┆ line_1  ┆ branch │
│ 3edce6fc-e164-5b3e-9b0d-8a1c86… ┆ line_2  ┆ branch │
│ c173d7f3-f7ca-5f44-98d8-63c1aa… ┆ line_4  ┆ branch │
│ c33afba9-0379-5d6a-9ca1-743cf7… ┆ line_5  ┆ branch │
│ 020bd95c-9320-5560-9627-4bbe19… ┆ line_6  ┆ branch │
│ …                               ┆ …       ┆ …      │
│ 34edd04e-a827-5cf8-b233-43d285… ┆ line_55 ┆ branch │
│ 9b34c19a-8b3e-54c6-9cd9-834bb5… ┆ line_56 ┆ branch │
│ ec28307e-8860-52db-9611-713cad… ┆ line_57 ┆ branch │
│ c8a03e72-a228-5007-ad70-3cc3ab… ┆ line_3  ┆ switch │
│ 4f44a759-906d-568d-95b7-a57f77… ┆ line_9  ┆ switch │
└─────────────────────────────────┴─────────┴────────┘


In [ ]:
node_id_list = (
    line_data.unpivot(on=["u_of_edge", "v_of_edge"], value_name="node_id")
    .unique("node_id", keep="first")["node_id"]
    .to_list()
)
node_id_to_nb_mapping = dict(zip(node_id_list, range(len(node_id_list))))
node_nb_to_id_mapping = dict(zip(range(len(node_id_list)), node_id_list))

line_data = line_data.with_columns(
    c(col).replace_strict(node_id_to_nb_mapping, default=None).alias(col)
    for col in ["u_of_edge", "v_of_edge"]
)
# ext_grid_nb = node_id_to_nb_mapping[ext_grid_id]

In [ ]:
# # from state_estimation_algorithm import StateEstimationData


# class StateEstimationData:
#     def __init__(self, changes_schema: dict):
#         self.line_data: pl.DataFrame
#         self.node_data: pl.DataFrame
#         self.changes_schema = changes_schema

#         self.generate_line_data()
#     ### TODO when code is ok
#     def generate_line_data(self):
#         self.line_data = self.changes_schema.resource.filter(
#             (c("concrete_class") == "branch") | (c("concrete_class") == "switch")
#         ).select(
#             c("uuid"), c("dso_code").alias("line_id"), c("concrete_class").alias("type")
#         )
# stateestimationdata = StateEstimationData(changes_schema=changes_schema)

In [ ]:
ext_grid_id = "1"
n = 0.95
v_ext_grid_sq = 1.05

line_data: pl.DataFrame = pl.DataFrame(
    {
        "line_id": np.arange(1, 13).astype(str),
        "type": ["branch"] * 10 + ["transformer"] * 2,
        "u_of_edge": ["2", "2", "2", "4", "5", "6", "7", "10", "12", "13", "8", "9"],
        "v_of_edge": ["11", "1", "3", "1", "4", "4", "8", "9", "11", "11", "4", "8"],
        "n_transfo": [1.0] * 10 + [n**2, n**2],
        "x_pu": np.arange(1, 13) * 2e-3,
        "r_pu": np.arange(1, 13) * 1e-3,
        "b_pu": list(np.arange(1, 11) * -1e-3) + [0.001, 0.001],
        "g_pu": [0.0] * 10 + [0.001, 0.001],
    }
)

node_data: pl.DataFrame = pl.DataFrame(
    {
        "node_id": np.arange(1, 14).astype(str),
        "v_base": [400.0] * 7 + [200.0, 400.0, 400.0, 200.0, 100.0, 100.0],
        "p_node_pu": np.array([0, 10, 2, 0, 0, -2, 0, 7, 25, 0, 100, 0.5, 10]) * 1e-2,
        "q_node_pu": np.array([0, 0, 0.2, 2, 0, -0.2, 0, 0.7, 0, 0, 0, 0.5, 1]) * 1e-2,
    },
    strict=False,
)

np.set_printoptions(linewidth=200)
distflow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)
node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sq = distflow.v_in_sq_np * v_ext_grid_sq

s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq, engine="numpy")
print("numpy:", time.time() - tic)

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sq=v0_sq, engine="graphblas"
)
print("graphblas:", time.time() - tic)

NameError: name 'DistFlow' is not defined

In [ ]:
s_base = 1e6
net: pp.pandapowerNet = pp.from_pickle("data/input_grid/modified_cigre_network_lv.p")
tic = time.time()
pp.runpp(net)
print("f:", time.time() - tic)

f: 0.012090921401977539


In [ ]:
ext_grid_id: str = str(net.ext_grid["bus"][0])
v_ext_grid_sq: float = net.ext_grid["vm_pu"][0] ** 2
node_data, line_data = pandapower_to_distflow(net=net, s_base=s_base)
distflow: DistFlow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)

In [ ]:
node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sq = distflow.v_in_sq_np * v_ext_grid_sq
s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq, engine="numpy")
print("numpy:", time.time() - tic)

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sq=v0_sq, engine="graphblas"
)
print("graphblas:", time.time() - tic)

numpy: 5.9482645988464355
graphblas: 0.09949016571044922


In [ ]:
line_res_pp = pl.DataFrame(
    {
        "node_id": list(net.line["to_bus"].astype(str)),
        "p_pp": list(net.res_line["p_from_mw"]),
        "q_pp": list(net.res_line["q_from_mvar"]),
    }
)

node_res_pp = pl.from_pandas(net.res_bus["vm_pu"], include_index=True).select(
    c("index").cast(pl.Utf8).alias("node_id"), "vm_pu"
)

result = (
    pl.DataFrame(
        {
            "v": np.sqrt(v),
            "p": np.real(s_flow),
            "q": np.imag(s_flow),
        }
    )
    .with_row_index(name="node_id")
    .with_columns(
        c("node_id")
        .replace_strict(distflow.node_nb_to_id_mapping, default=None)
        .alias("node_id")
    )
    .join(line_res_pp, on="node_id", how="left")
    .join(node_res_pp, on="node_id", how="left")
    .with_columns(
        ((c("v") - c("vm_pu")) / c("vm_pu")).abs().alias("diff_v"),
        ((c("p") - c("p_pp")) / c("p_pp")).abs().alias("diff_p"),
        ((c("q") - c("q_pp")) / c("p_pp")).abs().alias("diff_q"),
    )
)
print(result["diff_v", "diff_p", "diff_q"].max())

shape: (1, 3)
┌──────────┬──────────┬──────────┐
│ diff_v   ┆ diff_p   ┆ diff_q   │
│ ---      ┆ ---      ┆ ---      │
│ f64      ┆ f64      ┆ f64      │
╞══════════╪══════════╪══════════╡
│ 0.000392 ┆ 0.000182 ┆ 0.010306 │
└──────────┴──────────┴──────────┘


In [ ]:
net: pp.pandapowerNet = pp.from_pickle("data/input_grid/synthesized_grid.p")
net.trafo["i0_percent"] = 2
tic = time.time()
pp.runpp(net)
print("f:", time.time() - tic)

f: 0.5935556888580322


In [ ]:
s_base = 1e6
ext_grid_id: str = str(net.ext_grid["bus"][0])
v_ext_grid_sq: float = net.ext_grid["vm_pu"][0] ** 2

node_data, line_data = pandapower_to_distflow(net=net, s_base=s_base)
distflow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)

In [ ]:
node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sq = distflow.v_in_sq_np * v_ext_grid_sq
s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq, engine="numpy")
print("numpy:", time.time() - tic)

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sq=v0_sq, engine="graphblas"
)
print("graphblas:", time.time() - tic)

numpy: 9.821000337600708
graphblas: 0.08703160285949707


In [ ]:
line_data = line_data.group_by("u_of_edge", "v_of_edge").agg(
    c("name", "n_transfo", "type").first(),
    c("r_pu", "x_pu").first() / c("r_pu", "x_pu").count(),
    c("g_pu", "b_pu").first() * c("g_pu", "b_pu").count(),
)